In [ ]:
%%sql -r dataframe_1
USE DATABASE DBT_DEV_DB;
USE SCHEMA BRONZELAYER;

SELECT current_database() AS DATABASE_NAME,
 current_schema() AS SCHEMA_NAME;

In [ ]:
import logging
import pandas as pd
from datetime import datetime
from snowflake.snowpark.context import get_active_session

logger = logging.getLogger("bronze_ingest")
logger.setLevel(logging.INFO)

session = get_active_session()

# Get context directly from Snowflake
db = session.get_current_database()
schema = session.get_current_schema()

print(f"Running in: {db}.{schema}")
logger.info(f"Bronze ingestion started in {db}.{schema}")


In [ ]:
# =========================================
# Multi-CSV Bronze Ingestion (Fully Fixed)
# =========================================

from datetime import datetime
import pandas as pd
import os

STAGE_NAME = f"@{db}.{schema}.DEV_RAW_FILES_STAGE"

# Get stage file list
files_df = session.sql(f"LIST {STAGE_NAME}").to_pandas()

# Clean weird Snowflake quoted column names
files_df.columns = [c.replace('"', '').upper() for c in files_df.columns]

print("Cleaned columns:", files_df.columns)

# Extract CSV files
csv_files = [
    row["NAME"].split("/")[-1]
    for _, row in files_df.iterrows()
    if row["NAME"].lower().endswith(".csv")
]

print(f"Found {len(csv_files)} CSV files.")

for FILE_NAME in csv_files:

    TABLE_NAME = "RAW_" + os.path.splitext(FILE_NAME)[0].upper()

    print(f"\nProcessing: {FILE_NAME} → {TABLE_NAME}")

    session.file.get(
        f"{STAGE_NAME}/{FILE_NAME}",
        "/tmp/"
    )

    df = pd.read_csv(f"/tmp/{FILE_NAME}")

    df.columns = [
        col.strip().upper().replace(" ", "_").replace("-", "_")
        for col in df.columns
    ]

    df["_LOADED_AT"] = datetime.utcnow()
    df["_SOURCE_FILE"] = FILE_NAME

    snow_df = session.create_dataframe(df)
    snow_df.write.mode("append").save_as_table(f"{db}.{schema}.{TABLE_NAME}")

    row_count = session.table(f"{db}.{schema}.{TABLE_NAME}").count()

    print(f"Loaded {len(df)} rows. Table now has {row_count} rows.")

print("\nAll CSV files processed successfully.")
